In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from keras.layers import Input, Dense, Lambda, Flatten, Reshape
from keras.layers import Conv1D
from keras.models import Model
from keras import backend as K

n_features = 4
n_runs = 300
seq_length = [180, 360, 540, 720, 900]
labels = pd.read_csv("Data/Boat_nominal_data/Boat_mix_len_labels.csv")
labels = labels.drop(columns="Unnamed: 0") 
labels = np.array(labels)

def read_sequences():
    run_list_mix = []
    run_list_ordered = []
    labels_reordered = []
    for index in range(n_runs):
        run_csv = pd.read_csv("Mix_sequences_var_length/run^"+str(index))
        run_csv = run_csv.drop(columns=['Unnamed: 0'])
        run_list_mix.append(run_csv)
    for index, length in enumerate(seq_length):
        run_list_ordered.append([])
        for run_index, run in enumerate(run_list_mix): 
            if len(run) == length:
                run_list_ordered[index].append(run)    
                labels_reordered.append(labels[run_index])
    return run_list_ordered, labels_reordered
    
    
    
def stadard_sequences(sequences):
    for index, seqs in enumerate(sequences):
        for i, seq in enumerate(seqs):
            seqs[i] = MinMaxScaler(feature_range=[0, 1]).fit_transform(seq)
    return sequences                    


trainX_sequences, labels_ordered = read_sequences()
trainX_sequences = stadard_sequences(trainX_sequences)
for i, leng in enumerate(seq_length):
    trainX_sequences[i] = np.reshape(trainX_sequences[i], (-1, leng, n_features))

# def train_generator():
#     index = 0
#     while True:
#         x_train = trainX_sequences[index]
#         index += 1
#         yield (x_train, x_train) 


Using TensorFlow backend.


In [2]:

from keras.layers import Input, LSTM, RepeatVector, Conv2DTranspose
from keras.models import Model

filters = 50
latent_dim = 20

def Conv1DTranspose(input_tensor, filters, kernel_size,last, strides=1, padding='same'):
        if last:
            activation = 'linear'
        else:
            activation = 'relu'
        x = Lambda(lambda x: K.expand_dims(x, axis=2))(input_tensor)
        x = Conv2DTranspose(filters=filters, kernel_size=(kernel_size, 1), 
                            strides=(strides, 1), padding=padding,
                            activation=activation)(x)
        x = Lambda(lambda x: K.squeeze(x, axis=2))(x)
        return x


def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon


def repeat(x):
    stepMatrix = K.ones_like(x[0][:, :, :1])
    latentMatrix = K.expand_dims(x[1], axis=1)

    return K.batch_dot(stepMatrix, latentMatrix)


inputs = Input(shape=(None, n_features))
x = inputs
for i in range(3):
    x = Conv1D(filters=filters, kernel_size=2, strides=2,
               activation='relu', padding='same')(x)
   
x = LSTM(filters, return_sequences=True)(x)   
x_flat = LSTM(20)(x)

z_mean = Dense(latent_dim, name='z_mean')(x_flat)
z_log_var = Dense(latent_dim, name='z_log_var')(x_flat)
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

#encoded = Dense(latent_dim)(encoded)


x = Lambda(repeat)([inputs, z])

for i in range(3):
    x = Conv1DTranspose(input_tensor=x, filters=filters,
                        kernel_size=2, padding='same', last=False)
   
#decoded = LSTM(n_features, return_sequences=True)(x)

output = Dense(n_features)(x)

autoencoder = Model(inputs, output)
encoder = Model(inputs, [z_mean, z_log_var, z])


autoencoder.summary()
autoencoder.compile(optimizer='adam', loss='mse')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 4)      0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, None, 50)     450         input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, None, 50)     5050        conv1d_1[0][0]                   
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, None, 50)     5050        conv1d_2[0][0]                   
__________________________________________________________________________________________________
lstm_1 (LS

In [3]:
def train():
    for batch in trainX_sequences:
        if len(batch) != 0:
            autoencoder.fit(batch, batch, epochs=100, verbose=1)


train()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100



32/63 [==============>...............] - ETA: 4s - loss: 0.4148

63/63 [==============================] - 4s 71ms/step - loss: 0.3799


Epoch 2/100


...

32/55 [================>.............] - ETA: 0s - loss: 0.1134

55/55 [==============================] - 1s 24ms/step - loss: 0.1134


Epoch 100/100


32/55 [================>.............] - ETA: 0s - loss: 0.1134

55/55 [==============================] - 1s 23ms/step - loss: 0.1134


In [12]:
from sklearn.decomposition import PCA

encodings = []
for i,seqs in enumerate(trainX_sequences): 
    if len(seqs) != 0:
        encodings.append(encoder.predict(seqs))
    
print(len(encodings), len(encodings[0]), encodings[0][0].shape)

5 3 (63, 20)


In [11]:
def reshape_vectors():
    means, vars, z = [], [], []
    for encoded_seq in encodings:
        means.append(encoded_seq[0])
        vars.append(encoded_seq[1])
        z.append(encoded_seq[2])
    for i in encodings:
        for j in i:
            encodings_squeezed.append(j)
            encs = np.array(encodings_squeezed)
            print(encs.shape)
    
    return means, vars, z


means_encs, log_var_encs, z_encs = reshape_vectors()

print()

15 (63, 20)


In [7]:
def return_mask(num, labs):
    arg = np.squeeze(np.argwhere(labs == num))
    return arg

    
masks = [return_mask(num, np.array(labels_ordered))[:, 0] for num in range(0, 9)]
#print(labels_ordered - labels)
#masks_normal = [return_mask(num, np.array(labels))[:, 0] for num in range(0, 9)]


In [8]:

from mpl_toolkits.mplot3d import Axes3D  


def plot_pca(title, i): 
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    markers = ['o', 'o', 'o', 'o', '^', '^', '^', '^', '^', '^']
    for index, mask in enumerate(masks):
        ax.scatter(principalComponents[:, 0][mask], 
                   principalComponents[:, 1][mask],  
                   principalComponents[:, 2][mask], marker=markers[index])
        #break
    plt.legend(labels=np.arange(0, 9))
    plt.title(str(title))
    plt.show()
    
    for mask in masks:
        plt.scatter(x=principalComponents[:, 0][mask], 
                    y=principalComponents[:, 1][mask],
                    alpha=0.5)
        #break
    plt.legend(labels=np.arange(0, 9))
    plt.title(str(title))
    plt.show()
    

scaler = StandardScaler()
enc_input = scaler.fit_transform(encs) 
pca = PCA(3)
principalComponents = pca.fit_transform(enc_input)
print(principalComponents.shape)
print(pca.explained_variance_ratio_)
plot_pca('Sequences', 0)

principalComponents = encs
plot_pca('Sequences_Not_Pca', 0)



(300, 3)
[0.07521445 0.07078193 0.06656078]
